# **Exploratory Data Analysis**
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('dark_background')

## **1) Core Objectives**
---

### Overview:

Our primary objective with this exploratory analysis is to identify the features in our data that best represent our understanding of the [root causes of flight delays](../Research/flight_delay_reasons.md) and in so doing, provide the basis for the **accurate** and **reliable** prediction of commercial flight delays. We aim to develop a sense for how well our feature space captures common flight delay reasons in order to determine which features to select, which additional data to gather, and which features should be re-engineered or excluded.

### [Suggested Considerations](https://github.com/lighthouse-labs/mid-term-project-I/blob/master/exploratory_analysis.ipynb):

- Test the hypothesis that the delay is from Normal distribution and that mean of the delay is 0. Be careful about the outliers.
- Is average/median monthly delay different during the year? If so, which months have the biggest delays and what could be the reason?
- Does the weather affect the delay?
- How are taxi times changing during the day? Does higher traffic lead to longer taxi times?
- What is the average percentage of delays that exist prior to departure (*i.e.* are arrival delays caused by departure delays)? Are airlines able to lower the delay during the flights?
- How many states cover 50% of US air traffic?
- Test the hypothesis that planes fly faster when there is a departure delay.
- When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?
- Find the top 10 the bussiest airports. Does the greatest number of flights mean that the majority of passengers went through a given airport? How much traffic do these 10 airports cover?
- Do bigger delays lead to bigger fuel consumption per passenger?

## **2) Data Cleaning**
---

### **flights** ([sample](../Scripts/sampling.sql)) & **flights_test**

#### Overview:

##### **flights** sample

In [2]:
flights = pd.read_csv('../Data/files/flights_sample.csv')
flights.head()

M:\Anaconda\envs\LHL\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-05-19,UA,UA_CODESHARE,UA,4264,EV,N48901,4264,12266,IAH,...,127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-05-19,UA,UA_CODESHARE,UA,4266,EV,N12540,4266,13244,MEM,...,468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-05-19,UA,UA_CODESHARE,UA,4272,EV,N11164,4272,12266,IAH,...,1091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-05-19,UA,UA_CODESHARE,UA,4281,EV,N13995,4281,11042,CLE,...,310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-05-19,UA,UA_CODESHARE,UA,4286,EV,N13903,4286,13061,LRD,...,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2387955 entries, 0 to 2387954
Data columns (total 42 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fl_date              object 
 1   mkt_unique_carrier   object 
 2   branded_code_share   object 
 3   mkt_carrier          object 
 4   mkt_carrier_fl_num   int64  
 5   op_unique_carrier    object 
 6   tail_num             object 
 7   op_carrier_fl_num    int64  
 8   origin_airport_id    int64  
 9   origin               object 
 10  origin_city_name     object 
 11  dest_airport_id      int64  
 12  dest                 object 
 13  dest_city_name       object 
 14  crs_dep_time         int64  
 15  dep_time             float64
 16  dep_delay            float64
 17  taxi_out             float64
 18  wheels_off           float64
 19  wheels_on            float64
 20  taxi_in              float64
 21  crs_arr_time         int64  
 22  arr_time             float64
 23  arr_delay            float64
 24

##### **flights_test**

In [4]:
flights_test = pd.read_csv('../Data/files/flights_test.csv')
flights_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01 00:00:00,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01 00:00:00,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01 00:00:00,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01 00:00:00,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [5]:
flights_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   fl_date             660556 non-null  object
 1   mkt_unique_carrier  660556 non-null  object
 2   branded_code_share  660556 non-null  object
 3   mkt_carrier         660556 non-null  object
 4   mkt_carrier_fl_num  660556 non-null  int64 
 5   op_unique_carrier   660556 non-null  object
 6   tail_num            659057 non-null  object
 7   op_carrier_fl_num   660556 non-null  int64 
 8   origin_airport_id   660556 non-null  int64 
 9   origin              660556 non-null  object
 10  origin_city_name    660556 non-null  object
 11  dest_airport_id     660556 non-null  int64 
 12  dest                660556 non-null  object
 13  dest_city_name      660556 non-null  object
 14  crs_dep_time        660556 non-null  int64 
 15  crs_arr_time        660556 non-null  int64 
 16  du

#### **flights** sample vs. **flights_test** feature space

To establish the similarities and difference between the feature spaces of **flights** and **flights_test** rather quickly, we can leverage sets to find the intersection and symmetric difference of their features.

In [6]:
flights_initial_features = set(flights.columns)
flights_test_initial_features = set(flights_test.columns)

common_initial_features = flights_initial_features.intersection(flights_test_initial_features)
exclusive_initial_features = flights_initial_features.symmetric_difference(flights_test_initial_features)

Let us check if the flights_test feature space is entirely a subset of flights.

In [7]:
print(
    len(common_initial_features) == len(flights_test_initial_features), 
    len(flights_test_initial_features),
    sep="\n",
    )

True
20


This confirms that the feature space of flights_test is a subset of flights (which is expected given its [description](../Data/descriptions/flights_test_description.md)). Now let us check which additional features we have in the **flights** table.

In [8]:
print('Features exclusive to flights:\n')
for feature in exclusive_initial_features:
    print(feature)

print(f'\nNumber of features exclusive to flights: {len(exclusive_initial_features)}')

Features exclusive to flights:

late_aircraft_delay
first_dep_time
wheels_off
dep_delay
no_name
carrier_delay
diverted
cancellation_code
actual_elapsed_time
nas_delay
air_time
security_delay
arr_time
wheels_on
total_add_gtime
dep_time
taxi_out
arr_delay
cancelled
longest_add_gtime
weather_delay
taxi_in

Number of features exclusive to flights: 22


All of these features exclusive to **flights** are those that we can only know once the plane has already arrived (excluding `no_name`). Therefore, any useful features as part of this set would need to be predicted in order to make an inference about future flight delays.

#### Missing data investigation:

Preview missing data percentages for features with null values.

In [9]:
flights_missing_values = flights.isna().sum() > 0
flights_missing_value_percentage = flights.isna().sum() / flights.shape[0] * 100

flights_missing_value_percentage[flights_missing_values]

tail_num                 0.304947
dep_time                 1.616865
dep_delay                1.647979
taxi_out                 1.707151
wheels_off               1.707151
wheels_on                1.754681
taxi_in                  1.754681
arr_time                 1.718165
arr_delay                1.944132
cancellation_code       98.317305
crs_elapsed_time         0.000168
actual_elapsed_time      1.927172
air_time                 1.963605
carrier_delay           81.124770
weather_delay           81.124770
nas_delay               81.124770
security_delay          81.124770
late_aircraft_delay     81.124770
first_dep_time          99.313597
total_add_gtime         99.313722
longest_add_gtime       99.313681
no_name                100.000000
dtype: float64

In [10]:
flights_test_missing_values = flights_test.isna().sum() > 0
flights_test_missing_value_percentage = flights_test.isna().sum() / flights.shape[0] * 100

flights_test_missing_value_percentage[flights_test_missing_values]

tail_num    0.062773
dtype: float64

##### Empty rows

We will start by ensuring that empty rows are discarded as they offer no information.

In [11]:
flights = flights.dropna(how='all')
flights_test = flights_test.dropna(how='all')

##### `no_name`

no_name is completely blank, so we can drop it with absolutely no information loss.

In [12]:
flights = flights.drop('no_name', axis=1)

##### `cancellation_code`

In [13]:
flights.cancellation_code.unique()

array([nan, 'A', 'C', 'B', 'D'], dtype=object)

It may be worthwhile to investigate the `cancellation_code` feature further, but the missing proportion here should correspond with the `cancelled` feature assuming that codes are only present for cancelled flights (as suggested in the flights description). In other words, most data in this column is ***structually missing*** (i.e. missing because most flights were not cancelled).

In [14]:
flights['cancelled'].value_counts()/flights.shape[0] * 100

0    98.317305
1     1.682695
Name: cancelled, dtype: float64

The percentages support the structurally missing data hypothesis, but let us confirm by inspecting the relevant subset more rigorously.

In [15]:
flights[flights.cancelled > 0]['cancellation_code'].unique()

array(['A', 'C', 'B', 'D'], dtype=object)

In [16]:
flights[flights.cancelled == 0]['cancellation_code'].unique()

array([nan], dtype=object)

We have the most support that we can obtain from the data for the structurally missing hypothesis with respect to `cancellation_code`! It remains possible that human errors (e.g. data entry) are present but obscured in this table.

In [17]:
type(flights['cancellation_code'].unique()[0]) # nan is of type float in this case.

float

In [18]:
# Replace the missing value for now so that we do not consider the column further.
flights['cancellation_code'] = flights['cancellation_code'].apply(lambda code: 'none' if (isinstance(code, float)) else code)
flights['cancellation_code'].unique()

array(['none', 'A', 'C', 'B', 'D'], dtype=object)

##### '`_delay`' columns

All of the delay columns have identical percentages of missing data in **flights** (none are present in **flights_test**). Here we will test the hypothesis that the missing rows for all of these columns are the same. Resultantly, we predict that missing data will be consistent between rows for these features. This may suggest that these data are also ***structurally missing***.  

This may mean that any time a flight is delayed, all delay columns are assigned a baseline value of 0 (as opposed to NaN) and this baseline is incremented appropriately according to the class of delay.

In [19]:
delay_columns = ['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
delays_subframe = flights[delay_columns]
delays_subframe.head()
# These 5 rows concurrently displaying NaN provide very weak support for the above hypothesis. Let us investigate more closely.

,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


Let us check the rows that remain for each delay feature when those with missing values are discarded and ensure that it matches the value predicted given that our hypothesis is correct. Since we know that all delay columns have the same missing value percentage, we can use one as a reference value.

In [20]:
# Determine the actual percentages of non-missing data for the delay rows and the values we expect to see given our hypothesis.
delay_value_percentages = [(delays_subframe[column].dropna().count() / flights.shape[0] * 100)
                           for column in delays_subframe]
predicted_value = (100 - flights_missing_value_percentage.late_aircraft_delay)

np.allclose(delay_value_percentages, predicted_value) # used to prevent rounding errors from causing problems with strict equality

True

It does appear that the missing values correspond with no delay, and are thus structurally missing. It should be noted that the proportion of delayed flights in this sample lines up fairly well with what is observed overall in the publicly available data from the [US Department of Transportation](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?20=E). On this basis, we can impute these missing values as 0.

In [21]:
flights[delay_columns] = delays_subframe.fillna(0)

In [22]:
flights_missing_values = flights.isna().sum() > 0
flights_missing_value_percentage = flights.isna().sum() / flights.shape[0] * 100

flights_missing_value_percentage[flights_missing_values]

tail_num                0.304947
dep_time                1.616865
dep_delay               1.647979
taxi_out                1.707151
wheels_off              1.707151
wheels_on               1.754681
taxi_in                 1.754681
arr_time                1.718165
arr_delay               1.944132
crs_elapsed_time        0.000168
actual_elapsed_time     1.927172
air_time                1.963605
first_dep_time         99.313597
total_add_gtime        99.313722
longest_add_gtime      99.313681
dtype: float64

##### `dep_time` and `_gtime` columns

These columns are interesting because not only is the majority of data missing in similar percentages in all 3 of these columns, but the values are not precisely the same. Here we will investigate points of similarity and divergence between observations.

In [23]:
departure_and_gate_time_columns = ['first_dep_time', 'total_add_gtime', 'longest_add_gtime']
departure_and_gate_times = flights[departure_and_gate_time_columns]

In [24]:
some_missing = departure_and_gate_times.dropna(how='all')
no_missing = departure_and_gate_times.dropna()

missing_rows = set(some_missing.index).symmetric_difference(set(no_missing.index))
flights.loc[missing_rows].T # display the transpose for the 3 indices.

,813489,2020945,2027091
fl_date,2019-06-20,2018-02-26,2018-02-28
mkt_unique_carrier,B6,AS,DL
branded_code_share,B6,AS_CODESHARE,DL_CODESHARE
mkt_carrier,B6,AS,DL
mkt_carrier_fl_num,631,3296,5909
op_unique_carrier,B6,KS,YX
tail_num,N339JB,N682PA,N860RW
op_carrier_fl_num,631,3296,5909
origin_airport_id,10721,10299,12953
origin,BOS,ANC,LGA


In [25]:
flights.loc[missing_rows]['arr_delay']

813489     145.0
2020945    137.0
2027091    201.0
Name: arr_delay, dtype: float64

There are only 3 rows here with mixed missing values, and all have a departure time. Since none of these flights were cancelled, it is sensible to conclude that either none of them returned to the gate or these may be data entry errors (this is likely for index 2020945, which apparently never left the gate according to the 0 for `longest_add_gtime`).

Overall, because it is not clear what these features are adding and they are missing entirely for the overwhelming majority of the data, we will simply discard them.

In [26]:
flights = flights.drop(columns=departure_and_gate_time_columns)

##### Remainder of missing values:

Given the surplus of observations we have for the **flights** table, and the low missing data percentages for all other observations (<2%), it is worth checking how much data would be lost if all rows missing data were discarded.

In [31]:
total_observations_retained = flights.dropna().shape[0]
percent_data_retained = total_observations_retained / flights.shape[0] * 100

print(f'Total observations retained / {flights.shape[0]}: {total_observations_retained}')
print(f'Percentage retained: {percent_data_retained:.2f}')

Total observations retained / 2387955: 2339962
Percentage retained: 97.99


Given the minimal percentage loss in tandem with the massive sample size retained by dropping rows with missing values, the tradeoff it worth it in this instance.

In [32]:
flights = flights.dropna()

By the above logic, the tradeoff should be similarly valuable for **flights_test**:

In [33]:
total_observations_retained = flights_test.dropna().shape[0]
percent_data_retained = total_observations_retained / flights_test.shape[0] * 100

print(f'Total observations retained / {flights_test.shape[0]}: {total_observations_retained}')
print(f'Percentage retained: {percent_data_retained:.2f}')

Total observations retained / 660556: 659057
Percentage retained: 99.77


In [34]:
flights_test = flights_test.dropna()